# Sentiment Analysis

## Get the closing prices of Moderna

In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
%matplotlib inline

# Load .env environment variables
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# Format current date as ISO format
start_date = pd.Timestamp("2014-06-16", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-01-16", tz="America/New_York").isoformat()

# Set the tickers
ticker = "MRNA"

# Set timeframe to one day ('1D') for the Alpaca API
timeframe = "1D"

# Get current closing prices
df_closing_prices = alpaca.get_barset(
    ticker,
    timeframe,
    start = start_date,
    end = end_date
).df

# Dissolve multiindex and fetch the closing prices 
df_closing_prices = df_closing_prices.droplevel(0, axis=1)[['close']]

# Drop the time component of the date
df_closing_prices.index = df_closing_prices.index.date

# Display sample data
df_closing_prices.head()


Bad key "text.kerning_factor" on line 4 in
C:\Users\kn_na\anaconda3\envs\x\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


,close
2014-06-16,0.74
2014-06-17,0.70
2014-06-18,0.69
2014-06-19,0.69
2014-06-20,0.69


In [2]:
df_closing_prices['return'] = df_closing_prices['close'].pct_change()
df_closing_prices.dropna(inplace=True)
df_closing_prices.head()

,close,return
2014-06-17,0.700,-0.054054
2014-06-18,0.690,-0.014286
2014-06-19,0.690,0.000000
2014-06-20,0.690,0.000000
2014-06-23,0.625,-0.094203


## Load the data of google research results

In [3]:
import pandas as pd
from pathlib import Path
import calendar
from sklearn.preprocessing import LabelEncoder

file_path = Path("../companies_tweet_data/moderna.csv")
news_df = pd.read_csv(file_path, parse_dates=[['date','time']], infer_datetime_format=True, usecols=['date',"tweet",'time'])

# Delete the index label and sort in ascending order
news_df.set_index('date_time', inplace=True)
news_df.index.name = None
news_df.sort_index(axis=0, inplace=True)

# Display sample data
print(len(news_df))
display(news_df.head())

836


,tweet
2014-06-16 14:11:35,Moderna Announces Expansion to New Facility in...
2014-06-17 11:53:07,CNBC recognizes Moderna as a top 50 company wh...
2014-06-26 11:35:30,Boston Business Journal names Moderna's John R...
2014-06-29 19:15:14,@ge_og_mc We have a new account! Please follow...
2014-06-29 19:15:31,@ErikKiernan We have a new account! Please fol...


In [4]:
# Drop news published after 4pm since it does not affect the closing price of that day
news_df = news_df.between_time('00:00:00','23:59:59', include_end=False)

# Drop time in the index labels
news_df.index = news_df.index.date

# Display sample data
print(len(news_df))
display(news_df.head())

836


,tweet
2014-06-16,Moderna Announces Expansion to New Facility in...
2014-06-17,CNBC recognizes Moderna as a top 50 company wh...
2014-06-26,Boston Business Journal names Moderna's John R...
2014-06-29,@ge_og_mc We have a new account! Please follow...
2014-06-29,@ErikKiernan We have a new account! Please fol...


## Sentiment Analysis

In [5]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\kn_na\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
# Create the Facebook Libra sentiment scores DataFrame
sentiments = []

for title in news_df["tweet"]:
    try:
        sentiment = analyzer.polarity_scores(title) # get sentiment score
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        sentiments.append({"compound": compound,
                                 "positive": pos,
                                 "negative": neg,
                                 "neutral": neu
                                })
        
    except AttributeError:
        pass
    
# Create DataFrame
sentiments_df = pd.DataFrame(sentiments, index=news_df.index)

# Reorder DataFrame columns
news_df = news_df.join(sentiments_df)

news_df.head()

,tweet,compound,negative,neutral,positive
2014-06-16,Moderna Announces Expansion to New Facility in...,0.0000,0.0,1.000,0.000
2014-06-17,CNBC recognizes Moderna as a top 50 company wh...,0.2023,0.0,0.899,0.101
2014-06-26,Boston Business Journal names Moderna's John R...,0.0000,0.0,1.000,0.000
2014-06-29,@ge_og_mc We have a new account! Please follow...,0.3802,0.0,0.834,0.166
2014-06-29,@ge_og_mc We have a new account! Please follow...,0.3802,0.0,0.834,0.166


In [7]:
# sentiment_score_df = pd.DataFrame(index=news_df.index)
# sentiment_score_df.head()

In [8]:
# calculate the average of compound scores for a day with more than 1 article
import numpy as np
# sentiment_score_df = news_df.groupby(level=0)[['compound']].agg([('avg_sentiment',np.mean), ('article_counts',count)])
sentiment_score_df = news_df.groupby(level=0)[['compound']].count()
sentiment_score_df["avg_sentiments"] = news_df.groupby(level=0)[['compound']].mean()
sentiment_score_df.rename(columns= {'compound':"article_counts"}, inplace=True)
sentiment_score_df.head()

,article_counts,avg_sentiments
2014-06-16,1,0.0000
2014-06-17,1,0.2023
2014-06-26,1,0.0000
2014-06-29,1764,0.3802
2014-07-01,1,0.4767


## Concatenate the DFs of closing prices and sentiments

In [9]:
# concatenate
df = pd.concat([df_closing_prices, sentiment_score_df], axis=1, join='inner')

df.head()

,close,return,article_counts,avg_sentiments
2014-06-17,0.700,-0.054054,1,0.2023
2014-06-26,0.670,0.080645,1,0.0000
2014-07-01,0.620,-0.046154,1,0.4767
2014-07-08,0.595,-0.024590,1,0.4576
2014-07-25,0.560,0.000000,1,0.3612


## Prepare Training and Testing data

In [10]:
df["class"] = df["return"].apply(lambda x: 1 if x >= 0.01 else (0 if -0.01<x<0.01 else -1))

df.head(15)

,close,return,article_counts,avg_sentiments,class
2014-06-17,0.700,-0.054054,1,0.2023,-1
2014-06-26,0.670,0.080645,1,0.0000,1
2014-07-01,0.620,-0.046154,1,0.4767,-1
2014-07-08,0.595,-0.024590,1,0.4576,-1
2014-07-25,0.560,0.000000,1,0.3612,0
2014-07-28,0.560,0.000000,1,0.0000,0
2014-08-05,0.530,-0.121061,1,0.2960,-1
2014-08-06,0.590,0.113208,1,0.3818,1
2014-08-07,0.568,-0.037288,1,0.0000,-1
2014-08-08,0.575,0.012324,1,0.5859,1


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
            train_test_split(df[['article_counts','avg_sentiments']], df['class'], 
                             test_size=0.2,
                             random_state=1,
                             stratify=df['class']
                            )

X_train = X_train.values.reshape(-1,2)
X_test = X_test.values.reshape(-1,2)

## Choosing and fitting models

In [12]:
# Compare the five models and choose the best one
# Logistic regression
from sklearn.linear_model import LogisticRegression
algorithm1 = LogisticRegression(random_state=1)

# Support vector machine
from sklearn.svm import SVC
algorithm2 = SVC(kernel='rbf', random_state=1)

# Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
algorithm3 = DecisionTreeClassifier(random_state=1)

# Random Forest
from sklearn.ensemble import RandomForestClassifier
algorithm4 = RandomForestClassifier(random_state=1)

# XGBoost
from xgboost import XGBClassifier
algorithm5 = XGBClassifier(random_state=1)

# Create a list of the five model instances
algorithms = [algorithm1, algorithm2, algorithm3, algorithm4, algorithm5]

In [13]:
# Compare the performance of the five models
for algorithm in algorithms:
    algorithm.fit(X_train, y_train)
    score = algorithm.score(X_test, y_test)
    name = algorithm.__class__.__name__
    
    print(f'{name} score: {score:.4f}')

LogisticRegression score: 0.4054
SVC score: 0.4054
DecisionTreeClassifier score: 0.4324
RandomForestClassifier score: 0.3919


C:\Users\kn_na\anaconda3\envs\x\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:52:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier score: 0.4054


## Cross Validation

In [14]:
# Create a list of the five model instances
from sklearn.linear_model import LogisticRegression
algorithm1 = LogisticRegression(random_state=1)

from sklearn.svm import SVC
algorithm2 = SVC(kernel='rbf', gamma="scale", C=1, random_state=1)

from sklearn.tree import DecisionTreeClassifier
algorithm3 = DecisionTreeClassifier(random_state=1)

from sklearn.ensemble import RandomForestClassifier
algorithm4 = RandomForestClassifier(random_state=1)

from xgboost import XGBClassifier
algorithm5 = XGBClassifier(random_state=1)

algorithms = [algorithm1, algorithm2, algorithm3, algorithm4, algorithm5]

In [15]:
# Compare the performance of the five models

# Create a balanced set of samples, create a StratifiedKFold instance
from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

from sklearn.model_selection import cross_val_score

for algorithm in algorithms:
    # Conduct cross validation for each one of the five models
    scores = cross_val_score(algorithm, X_train, y_train, cv=stratifiedkfold)
    score = scores.mean()
    name = algorithm.__class__.__name__
    print(f'{name} average score: {score:.4f} / each score: {scores}')

LogisticRegression average score: 0.4077 / each score: [0.36734694 0.45360825 0.40206186]
SVC average score: 0.4075 / each score: [0.40816327 0.41237113 0.40206186]
DecisionTreeClassifier average score: 0.3767 / each score: [0.3877551  0.35051546 0.39175258]
RandomForestClassifier average score: 0.3629 / each score: [0.3877551  0.3814433  0.31958763]
[20:52:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\kn_na\anaconda3\envs\x\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:52:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:52:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier average score: 0.3698 / each score: [0.3877551  0.39175258 0.32989691]


## Grid Search

In [16]:
# Optimize the parameters
params = {'C':[1, 10, 100, 1000, 10000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001, 0.00001]}
algorithm = SVC(random_state=1)

from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(algorithm, params, cv=stratifiedkfold)
gs.fit(X_train, y_train)

# Based on the best parameters, predict y values from test data
best = gs.best_estimator_
best_pred = best.predict(X_test)
print(best)

SVC(C=100, gamma=1, random_state=1)


In [17]:
# Accuracy
score = best.score(X_test, y_test)
print(f'score: {score:.4f}')

# Confusion matrix
from sklearn.metrics import confusion_matrix
print('confusion matrix')
print(confusion_matrix(y_test, best_pred))

score: 0.4054
confusion matrix
[[11  0 17]
 [ 8  0  8]
 [10  1 19]]
